In [228]:
#imports 
import pandas as pd
from datetime import datetime as dt
import numpy as np
import difflib as dif
import re
import os
import math


# Convert MP CSV to Edlink Format

In [296]:
# MindPrint csv specific
class_columns = ['class_1', 'class_2', 'class_3', 'class_4']

In [297]:
#edlink types
roles = ['student','district-administrator','administrator','teacher']

def format_fix (current_value, acceptable_values ):
    if current_value not in acceptable_values:
        matches = dif.get_close_matches(current_value, acceptable_values, n = 1, cutoff = 0.7)

        if(len(matches) > 0):
            replacement = matches[0]
            # print(current_value, "not found in acceptable matches, replacing with", replacement)
            return replacement
        else:
            print("No match found for", current_value, ", please edit the csv")

    return current_value

In [298]:
df = pd.read_csv("master_roster.csv")
df

,first_name,last_name,password,email,role,gender,birthdate,user_school_id,school_name,grade,class_1,class_2,class_3,class_4
0,Sam,Bobrick,org_admin,Bobrick@bayside.demo,District_Administrator,NaN,NaN,18400,NaN,NaN,NaN,NaN,NaN,NaN
1,Michael,Poryes,org_admin,Poryes@bayside.demo,District_Administrator,NaN,NaN,18401,NaN,NaN,NaN,NaN,NaN,NaN
2,Susan,Sherman,org_admin,Sherman@bayside.demo,District_Administrator,NaN,NaN,18402,NaN,NaN,NaN,NaN,NaN,NaN
3,Richard,Belding,site_admin,Belding@bayside.demo,School_Administrator,NaN,NaN,18403,Bayside High School,NaN,NaN,NaN,NaN,NaN
4,Wesley,Lawler,site_admin,Lawler@bayside.demo,School_Administrator,NaN,NaN,18404,Bayside Middle School,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Larissa,Swing,student,Swing@bayside.demo,Student,female,2016-09-30,18469,Bayside Elementary School,3.0,Grade 03 - Depaolo,NaN,NaN,NaN
69,Augustin,Carbonelle,student,Carbonelle@bayside.demo,Student,male,2016-11-12,18470,Bayside Elementary School,3.0,Grade 03 - Depaolo,NaN,NaN,NaN
70,Vivian,Shah,student,Shah@bayside.demo,Student,female,2016-02-15,18471,Bayside Elementary School,3.0,Grade 03 - Depaolo,NaN,NaN,NaN
71,David,Langhoff,student,Langhoff@bayside.demo,Student,male,2016-04-27,18472,Bayside Elementary School,3.0,Grade 03 - Depaolo,NaN,NaN,NaN


In [299]:
schools = pd.DataFrame(df["school_name"].unique())
schools = schools.dropna(how="any")
schools.insert(0, 'id', range(0, 0 + len(schools)))
schools = schools.rename(columns={0:"school_name"})
schools.to_csv("schools.csv", index=False)
schools

,id,school_name
1,0,Bayside High School
2,1,Bayside Middle School
3,2,Bayside Elementary School


In [300]:
classes = []
for c in class_columns:
    for cl in df[c].unique():
        classes.append(cl)

classes = pd.DataFrame(classes)
classes = classes.dropna(how='any')
classes = classes.drop_duplicates()
classes = classes.rename(columns={0:"class_name"})
classes.insert(0, 'id', range(2000, 2000 + len(classes)))
classes.to_csv("classes.csv", index=False)
classes

,id,class_name
1,2000,MATH-11A
2,2001,ELA-11B
3,2002,ART-11C
4,2003,SCIENCE-11C
5,2004,SPANISH-6A
6,2005,HISTORY-6D
7,2006,THEATRE-6A
8,2007,Grade 03 - Depaolo


In [301]:
def convert_to_id(type, name, data):
    if pd.isnull(name):
        return 
    s = data.set_index(type)['id']
    return s[name]

result = convert_to_id("class_name", "THEATRE-6A", classes)
result

2006

### Convert dataframe values to id counterparts

In [302]:
df['school_name'] = df.apply(lambda x: convert_to_id('school_name', x['school_name'], schools),axis=1)
df = df.rename(columns={'school_name': 'school_id'})

df['role'] = df.apply(lambda x: format_fix(x['role'], roles),axis=1)
df.insert(0, 'id', range(0, 0 + len(df)))

for c in class_columns:
    df[c] = df.apply(lambda x: convert_to_id('class_name', x[c], classes),axis=1)

### Create class array column

In [303]:
class_array = df[(list(df.filter(regex='class')))]
class_array = class_array.values
class_array

clean_class_arrays = []
for array in class_array:
    array = array[~np.isnan(array)]
    clean_class_arrays.append(array)

df["classes"] = clean_class_arrays
df

,id,first_name,last_name,password,email,role,gender,birthdate,user_school_id,school_id,grade,class_1,class_2,class_3,class_4,classes
0,0,Sam,Bobrick,org_admin,Bobrick@bayside.demo,district-administrator,NaN,NaN,18400,NaN,NaN,NaN,NaN,NaN,NaN,[]
1,1,Michael,Poryes,org_admin,Poryes@bayside.demo,district-administrator,NaN,NaN,18401,NaN,NaN,NaN,NaN,NaN,NaN,[]
2,2,Susan,Sherman,org_admin,Sherman@bayside.demo,district-administrator,NaN,NaN,18402,NaN,NaN,NaN,NaN,NaN,NaN,[]
3,3,Richard,Belding,site_admin,Belding@bayside.demo,administrator,NaN,NaN,18403,0.0,NaN,NaN,NaN,NaN,NaN,[]
4,4,Wesley,Lawler,site_admin,Lawler@bayside.demo,administrator,NaN,NaN,18404,1.0,NaN,NaN,NaN,NaN,NaN,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,68,Larissa,Swing,student,Swing@bayside.demo,student,female,2016-09-30,18469,2.0,3.0,2007.0,NaN,NaN,NaN,[2007.0]
69,69,Augustin,Carbonelle,student,Carbonelle@bayside.demo,student,male,2016-11-12,18470,2.0,3.0,2007.0,NaN,NaN,NaN,[2007.0]
70,70,Vivian,Shah,student,Shah@bayside.demo,student,female,2016-02-15,18471,2.0,3.0,2007.0,NaN,NaN,NaN,[2007.0]
71,71,David,Langhoff,student,Langhoff@bayside.demo,student,male,2016-04-27,18472,2.0,3.0,2007.0,NaN,NaN,NaN,[2007.0]


In [304]:
def get_enrollments(user_enrollments, user_id, classes):
    for c in classes:
        user_enrollments.append([user_id, c])
    
    return user_enrollments

In [305]:
enrollment_data = df[["id", "classes"]]
enrollments = []

enrollment_data.apply(lambda x: get_enrollments(enrollments, x['id'], x["classes"]),axis=1)

enrollments = pd.DataFrame(enrollments)
enrollments = enrollments.rename(columns={0: "person_id", 1: "class_id"})
enrollments.insert(0, 'id', range(5000, 5000 + len(enrollments)))
enrollments.to_csv("enrollments.csv", index=False)
enrollments

,id,person_id,class_id
0,5000,6,2000.0
1,5001,7,2001.0
2,5002,8,2002.0
3,5003,9,2003.0
4,5004,10,2000.0
...,...,...,...
171,5171,68,2007.0
172,5172,69,2007.0
173,5173,70,2007.0
174,5174,71,2007.0


In [306]:
users = df.drop(columns=(list(df.filter(regex='class'))))
users.to_csv("users.csv", index=False)
users

,id,first_name,last_name,password,email,role,gender,birthdate,user_school_id,school_id,grade
0,0,Sam,Bobrick,org_admin,Bobrick@bayside.demo,district-administrator,NaN,NaN,18400,NaN,NaN
1,1,Michael,Poryes,org_admin,Poryes@bayside.demo,district-administrator,NaN,NaN,18401,NaN,NaN
2,2,Susan,Sherman,org_admin,Sherman@bayside.demo,district-administrator,NaN,NaN,18402,NaN,NaN
3,3,Richard,Belding,site_admin,Belding@bayside.demo,administrator,NaN,NaN,18403,0.0,NaN
4,4,Wesley,Lawler,site_admin,Lawler@bayside.demo,administrator,NaN,NaN,18404,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
68,68,Larissa,Swing,student,Swing@bayside.demo,student,female,2016-09-30,18469,2.0,3.0
69,69,Augustin,Carbonelle,student,Carbonelle@bayside.demo,student,male,2016-11-12,18470,2.0,3.0
70,70,Vivian,Shah,student,Shah@bayside.demo,student,female,2016-02-15,18471,2.0,3.0
71,71,David,Langhoff,student,Langhoff@bayside.demo,student,male,2016-04-27,18472,2.0,3.0
